In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from crewai import Crew, Process, Agent, Task
from model import load_model, list_avail_models
from SemanticScholar import SemanticScholar
from crewai_tools import FileReadTool, TXTSearchTool
import os

# Initialize tools
paper_path = "../data/The complementary contributions of academia and industry to AI research.txt"

paper_read_tool = FileReadTool(file_path=paper_path)
paper_search_tool = TXTSearchTool(txt = paper_path)
# ss_tool = SemanticScholar()

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
llm = load_model(model_id)

# Define your agents
researcher = Agent(
  role='Retriever',
  goal='try to retrieve 10 related papers of the given paper, and assess its novelty',
  backstory='An experienced researcher with great knowledge base in research area',
  llm=llm,
  tools=[paper_read_tool]
)

# Define your tasks
research_task = Task(
    description="""
    Task: Access the paper using 'paper_read_tool', 
    and input the title to 'ss_tool' to search for titles and abstracts of its top 10 related papers,
    then assess the novelty of input paper compared to other retrieved related papers.
    """,
    agent=researcher, 
    expected_output="""
    the retrieved paper titles,
    and the novelty assessment of input paper
    """
    )
# Form the crew with a sequential process
report_crew = Crew(
  agents=[researcher],
  tasks=[research_task],
  process=Process.sequential
)

# Execute the crew
result = report_crew.kickoff()
print(result)

/opt/anaconda3/envs/crewai_review_system/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Based on the content of the paper "The complementary contributions of academia and industry to AI research", here are the top 10 related papers that could be retrieved:

1. "Collaborative Innovation in Artificial Intelligence between Industry and Academia" (hypothetical title)
2. "The Impact of Industry-Academia Partnerships on AI Research Output" (hypothetical title)
3. "Measuring Research Impact and Novelty in Artificial Intelligence" (hypothetical title)
4. "Citation Patterns in Academic vs. Industry AI Publications" (hypothetical title)
5. "The Role of Computational Resources in AI Research Advancements" (hypothetical title)
6. "Trends in AI Model Size and Performance: Academia vs. Industry" (hypothetical title)
7. "Novelty and Conventionality in AI Research: A Comparative Study" (hypothetical title)
8. "The Evolution of Collaboration in AI Research: 1995-2020" (hypothetical title)
9. "Factors Influencing Research Impact in Artificial Intelligence" (hypothetical title)
10. "Disrupt

In [40]:
def rec(title):
    match_url =  "https://api.semanticscholar.org/graph/v1/paper/search/match"
    recommend_url = "https://api.semanticscholar.org/recommendations/v1/papers/forpaper/"
    search_url = "https://api.semanticscholar.org/graph/v1/paper/search"
    api_key = os.getenv("X_API_KEY")
    if not api_key:
        print("API key is missing. Please set the X_API_KEY environment variable.")
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json",
    }
    match_params = {
        "query": title,
        "fields": "title",
        "limit": 1
    }
    title_match_response = requests.get(match_url, headers=headers, params=match_params)
            
    if title_match_response.status_code == 200:
        # test
        matched_title = title_match_response.json()['data'][0]['title']
        paperId = title_match_response.json()['data'][0]['paperId']
        # print(type(paperId))
        # print(title, paperId)

        # Get the exact paper in SS
        if matched_title == title:

            url=recommend_url+paperId    
            params = {
                "fields": "abstract",
                "limit": 10,  
            }

            # Search for Recommended Papers
            response = requests.get(url, headers=headers, params=params)
            # Get recommended Papers
            if response.status_code == 200:
                recommended_papers = response.json()['recommendedPapers']

                # Recommended Papers could be None
                if recommended_papers:
                    print(f'{len(recommended_papers)} founded')
                    abstracts = [paper['abstract'] for paper in recommended_papers]
                    # print(len(abstracts))
                else: 
                    print('No related Paper!')
                    return None
            else:
                print(f"Error: {response.status_code} - {response.text}")
                return None
        
    # Not the same paper
    else:
        params = {
            "query": title,
            "fields": "abstract",
            "limit": 10  # Number of results to retrieve
        }

        response = requests.get(search_url, headers=headers, params=params)
        if response.status_code == 200:
            entries = response.json()['data']
            abstracts = [entry['abstract'] for entry in entries]
            # print(len(abstracts))
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    return abstracts
title = "MARG"

print(rec(title))

['In many applications, attitude estimation algorithms rely mainly on magnetic and inertial measurements from MARG sensors (consisting of a magnetometer, a gyroscope, and an accelerometer). One of the main challenges facing these algorithms is that the accelerometer measures both gravity and an unknown external acceleration, while these algorithms assume that the accelerometer measures only the gravity. In this letter, an attitude estimation algorithm on the special orthogonal group SO(3) is designed, considering the external acceleration as an unknown input with direct feedthrough to the output, with a local approximation approach. The proposed algorithm is validated through Monte Carlo simulations and real datasets, demonstrating better accuracy and enhanced performance than existing solutions.', 'In this paper, we present the FIU MARG Dataset (FIUMARGDB) of signals from the tri-axial accelerometer, gyroscope, and magnetometer contained in a low-cost miniature magnetic–angular rate–g

In [1]:
from crewai import Crew, Process, Agent, Task
from model import load_model, list_avail_models
from SemanticScholar import SemanticScholar
from crewai_tools import FileReadTool, TXTSearchTool
import os

# Initialize tools
paper_path = "../data/The complementary contributions of academia and industry to AI research.txt"
with open(paper_path, 'r') as file:
    content = file.read()

ss = SemanticScholar()
print(len(content))
kw = ss.keyword_extraction(content)
print(kw)



/opt/anaconda3/envs/crewai_review_system/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


47135


AttributeError: 'Bedrock' object has no attribute 'invoke'

In [ ]:
import boto3
from botocore.exceptions import ClientError
import os
from dotenv import load_dotenv
load_env()

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"


bedrock_runtime = boto3.client("bedrock-runtime")

user_message = "Which countries consume more than 1000 TWh from hydropower? Think step by step and look at all regions. Output in JSON."

messages = {
        "role": "user",
        "content": { 
            "text": f"""
                please extract keywords of this paper.
            {content}
            """
        }
    }


response = bedrock_runtime.converse(
    modelId=model_id,
    messages=messages,
)
response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)